In [2]:
import dcamcon

In [3]:
import dcam

In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 2
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            if (not signaled_sigint and
                res):
                res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L







Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a fram

In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 2
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            if (not signaled_sigint and
                res):
                res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L







Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 2
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=True):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
DCAM-API uninit'd..
End dcamcon_live_capturing.py


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 2
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to ar

In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, True)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    print(frameinterval)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
0.009997714285714285
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    # print(frameinterval)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    print(timeout_millisec)
    
    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
519
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 2000
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    # print(frameinterval)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    print(timeout_millisec)

    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
519
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.


In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    # print(frameinterval)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 1000
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    print(timeout_millisec)

    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
1019
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.


In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(1) is not False:
                if dcam.cap_snapshot() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()


if __name__ == '__main__':
    dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(1) is not False:
                if dcam.cap_snapshot() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()


if __name__ == '__main__':
    dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(1) is not False:
                if dcam.cap_snapshot() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()


if __name__ == '__main__':
    dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()


if __name__ == '__main__':
    dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
len(data_list)

TypeError: object of type 'NoneType' has no len()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
len(data_list)

1

In [ ]:
data_list[0]

array([[101, 104,  99, ..., 106,  99,  99],
       [101, 106, 102, ..., 103, 106, 101],
       [103, 100, 100, ..., 106,  98, 100],
       ...,
       [101, 109, 105, ...,  96, 119, 108],
       [132, 110, 106, ...,  97, 103, 106],
       [100, 106,  93, ..., 108, 101, 100]], dtype=uint16)

In [ ]:
data_list[0].shape

(2048, 2048)

In [ ]:
import cv2
cv2.imshow(data_list[0])

TypeError: imshow() missing required argument 'mat' (pos 2)

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
len(data_list)

1

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            # break

                        dcamerr = dcam.lasterr()
                        if dcamerr.is_timeout():
                            print('===: timeout')
                            continue

                        print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

-NG: Dcam.wait_event() fails with error 1


In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start() is not False:
                    timeout_milisec = 1000
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
len(data_list)

10

In [ ]:
for i in range(0,10):
    print(data_list[i].shape)


(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)


In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            if (time_now-start_time>20):
                                break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for i in range(0,len(data_list)):
    cv2.imshow('image', data_list[i])
    cv2.waitKey(1)

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            if (time_now-start_time>20):
                                break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

: 

In [ ]:
for i in range(0,len(data_list)):
    cv2.imshow('image', data_list[i])
    cv2.waitKey(1)

NameError: name 'data_list' is not defined

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            if (time_now-start_time>20):
                                break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for i in range(0,len(data_list)):
    cv2.imshow('image', data_list[i])
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            if (time_now-start_time>5):
                                break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=True) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    while True:
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            if (time_now-start_time>5):
                                break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(100) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(100) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,100):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(1000) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,100):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcam import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(500) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,500):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
#%%

__date__ = '2023-12-06'
__copyright__ = 'Copyright (C) 2024 Hamamatsu Photonics K.K.'

import os
# for getting file name form __file__

import signal
# for handiling pressing Ctrl+C

import cv2
# pip install opencv-python
# for disply image
# tested with (4.5.1.48) which used numpy (1.20.2)

from screeninfo import get_monitors
# pip install screeninfo
# for getting monitor information

from dcamcon import *
# for control DCAM fucntion

# OpenCV window status.
# 0 = not created yet
# 1 = already created and open
# -1 = close manually by user 
cv_window_status = 0

signaled_sigint = False    # True means Ctrl+C was pressed

# if True, acquired images is saved as .RAW at finished live.
save_at_finish_live = False

def setup_properties(dcamcon):
    """Set basic properties.
    Set basic properties for capturing.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    
    Returns:
        bool: result
    """
    # restrict pixeltype due to opencv
    restrict_pixeltype = [DCAM_PIXELTYPE.MONO8,
                          DCAM_PIXELTYPE.MONO16]
    # restrict triggersource. standard trigger only.
    # excluding triggers that require individual settings  
    restrict_triggersource = [DCAMPROP.TRIGGERSOURCE.INTERNAL,
                              DCAMPROP.TRIGGERSOURCE.EXTERNAL,
                              DCAMPROP.TRIGGERSOURCE.SOFTWARE]
    
    # set properties
    if (dcamcon.prompt_propvalue(DCAM_IDPROP.SENSORMODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.READOUTSPEED) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.IMAGE_PIXELTYPE, PromptRestrictMode.ModeFilter, restrict_pixeltype) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERSOURCE, PromptRestrictMode.ModeFilter, restrict_triggersource) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGER_MODE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERACTIVE) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.TRIGGERPOLARITY) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.BINNING) is False or
        # propmpt DCAM_IDPROP.SUBARRAYMODE, DCAM_IDPROP.SUBARRAYHPOS,
        # DCAM_IDPROP.SUBARRAYVPOS and DCAM_IDPROP.SUBARRAYVSIZE
        dcamcon.prompt_propvalue_subarray() is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.EXPOSURETIME) is False or
        dcamcon.prompt_propvalue(DCAM_IDPROP.CONTRASTGAIN) is False):
            return False
    
    return True

def show_framedata(camera_title, data):
    """Show image data.
    Open window of OpenCV with camera_title.
    Show numpy buffer as an image with OpenCV.

    Args:
        camera_title (string): for OpenCV window title
        data (Numpy ndarray): numpy buffer stored image
    """
    global signaled_sigint
    if signaled_sigint:
        return
    
    global cv_window_status
    if cv_window_status > 0:    # was the window created and open?
        cv_window_status = cv2.getWindowProperty(camera_title, 0)
        if cv_window_status == 0:   # if it is still open
            cv_window_status = 1    # mark it as still open again
    
    if cv_window_status >= 0:    # see if the window is not created yet or created and open
        maxval = np.amax(data)
        if data.dtype == np.uint16:
            if maxval > 0:
                imul = int(65535 / maxval)
                data = data * imul
        
        if cv_window_status == 0:
            # OpenCV window is not created yet
            cv2.namedWindow(camera_title, cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO | cv2.WINDOW_GUI_NORMAL)

            # resize display window
            data_width = data.shape[1]
            data_height = data.shape[0]

            window_pos_left = 156
            window_pos_top = 48

            screeninfos = get_monitors()

            max_width = screeninfos[0].width - (window_pos_left * 2)
            max_height = screeninfos[0].height - (window_pos_top * 2)

            if data_width > max_width:
                scale_X100 = int(100 * max_width / data_width)
            else:
                scale_X100 = 100
            
            if data_height > max_height:
                scale_Y100 = int(100 * max_height / data_height)
            else:
                scale_Y100 = 100
            
            if scale_X100 < scale_Y100:
                scale_100 = scale_X100
            else:
                scale_100 = scale_Y100
            
            disp_width = int(data_width * scale_100 * 0.01)
            disp_height = int(data_height * scale_100 * 0.01)

            cv2.resizeWindow(camera_title, disp_width, disp_height)
            # end of resize

            cv2.moveWindow(camera_title, window_pos_left, window_pos_top)
            cv_window_status = 1
        
        cv2.imshow(camera_title, data)
        key = cv2.waitKey(1)
        if key == ord('q') or key == ord('Q'):  # if 'q' or 'Q' was pressed with the live window, close it
            cv_window_status = -1
        

def show_live_captured_images(dcamcon: Dcamcon):
    """Show live images.
    It captures images and shows live images.

    Args:
        dcamcon (Dcamcon): Dcamcon instance
    """
    # get property value used
    exposuretime = dcamcon.get_propertyvalue(DCAM_IDPROP.EXPOSURETIME)
    if exposuretime is False:
        # should be able to get the value
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)
    if triggersource is False:
        # should be able to get the value
        return
    
    trigger_mode = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGER_MODE)
    if trigger_mode is False:
        # should be able to get the value
        return


    number_of_frames = 10
    # prepare buffer
    if not dcamcon.allocbuffer(number_of_frames):
        return
    
    # calculate timeout time
    timeout_millisec = 200
    
    frameinterval = dcamcon.get_propertyvalue(DCAM_IDPROP.INTERNAL_FRAMEINTERVAL, False)
    # print(frameinterval)
    if frameinterval is not False:
        # set timeout waiting for a frame to arrive to exposure time + internal frame interval + 500 ms
        timeout_millisec = int((exposuretime + frameinterval) * 1000.0) + 500
    else:
        # set timeout waiting for a frame to arrive to exposure time + 1 second
        timeout_millisec = int(exposuretime * 1000.0) + 1000
    
    timeout_millisec=1000
    # let's use 2ms minimum timeout
    if timeout_millisec < 2:
        timeout_millisec = 2
    print(timeout_millisec)

    # start live
    if not dcamcon.startcapture(is_sequence=False):
        # dcamcon.allocbuffer() should have succeeded
        dcamcon.releasebuffer()
        return
    
    triggersource = dcamcon.get_propertyvalue(DCAM_IDPROP.TRIGGERSOURCE)

    firetrigger_cycle = 0
    framecount_till_firetrigger = 0
    if triggersource == DCAMPROP.TRIGGERSOURCE.SOFTWARE:
        if trigger_mode == DCAMPROP.TRIGGER_MODE.START:
            # Software Start requires only one firetrigger at beginning
            firetrigger_cycle = 0
        elif trigger_mode == DCAMPROP.TRIGGER_MODE.PIV:
            # PIV require firetrigger for 2 frames
            firetrigger_cycle = 2
        else:
            # standard software trigger requires one firetrigger for one frame
            firetrigger_cycle = 1
        
        # we'll fire a trigger to initiate capturing for this sample
        dcamcon.firetrigger()
        framecount_till_firetrigger = firetrigger_cycle
    
    timeout_happened = 0

    global cv_window_status
    global signaled_sigint
    while cv_window_status >= 0:
        if signaled_sigint:
            break

        res = dcamcon.wait_capevent_frameready(timeout_millisec)
        if res is not True:
            # frame does not come
            if res != DCAMERR.TIMEOUT:  # note the != comparison
                print('-NG: Dcam.wait_event() failed with error {}'.format(res))
                break

            # TIMEOUT error happens
            timeout_happened += 1
            if timeout_happened == 1:
                print('Waiting for a frame to arrive.', end='')
                if triggersource == DCAMPROP.TRIGGERSOURCE.EXTERNAL:
                    print(' Check your trigger source.', end ='')
                else:
                    print(' Check your <timeout_millisec> calculation in the code.', end='')
                print(' Press Ctrl+C to abort.')
            else:
                print('.')
                if timeout_happened > 5:
                    timeout_happened = 0
            
            continue

        # wait_capevent_frameready() succeeded
        lastdata = dcamcon.get_lastframedata()
        if lastdata is not False:
            show_framedata(dcamcon.device_title, lastdata)
        
        if framecount_till_firetrigger > 0:
            framecount_till_firetrigger -= 1
            if framecount_till_firetrigger == 0:
                dcamcon.firetrigger()
                framecount_till_firetrigger = firetrigger_cycle
        
        timeout_happened = 0

    # End live
    cv2.destroyAllWindows()
    
    dcamcon.stopcapture()

    if save_at_finish_live:
        dcamcon.save_rawimages('LastGoodImage')

    dcamcon.releasebuffer()

def sigint_handler(signum, frame):
    """Detect pressing Ctrl+C.
    Signal handler function.
    This script is handling SIGINT only.
    signaled_sigint is set True.
    """
    global signaled_sigint
    signaled_sigint = True

#run handler (which does cleanup) if Ctrl+C is pressed in Python console.
signal.signal(signal.SIGINT, sigint_handler)

if __name__ == '__main__':
    ownname = os.path.basename(__file__)
    print('Start {}'.format(ownname))

    # initialize DCAM-API
    if dcamcon_init():
        # choose camera and get Dcamcon instance
        dcamcon = dcamcon_choose_and_open()
        if dcamcon is not None:
            res = True
            # set basic properties
            # if (not signaled_sigint and
            #     res):
            #     res = setup_properties(dcamcon)
            
            # show live image
            if (not signaled_sigint and
                res):
                show_live_captured_images(dcamcon)
            
            # close dcam
            dcamcon.close()
    
    # cleanup dcamcon
    dcamcon_uninit()

    print('End {}'.format(ownname))

Start dcamcon_live_capturing.py
Calling Dcamapi.init()
DCAM-API Init'd..
#[0]: MODEL=C13440-20C, CAMERAID=S/N: 303200, BUS=AS-FBD-1XCLD-2PE4L
1000
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame to arrive. Check your <timeout_millisec> calculation in the code. Press Ctrl+C to abort.
.
.
.
.
.
Waiting for a frame 

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcamcon import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            dcam.prop_setvalue(DCAM_IDPROP.TRIGGER_MODE)
            if dcam.buf_alloc(500) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,500):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

TypeError: prop_setvalue() missing 1 required positional argument: 'fValue'

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcamcon import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

-NG: Dcamapi.init() fails with error -520093695


In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcamcon import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(10) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,10):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
#%%

__date__ = '2021-06-18'
__copyright__ = 'Copyright (C) 2021-2024 Hamamatsu Photonics K.K.'

import cv2, time
# pip install opencv-python
# - tested with (4.5.1.48) which used numpy (1.20.2)
# - used for the display

from dcamcon import *
# control DCAM functions

def dcamtest_show_framedata(data):
    """Show image data.
    Show numpy buffer as an image with OpenCV function.

    Args:
        data (void): NumPy array.

    Returns:
        Nothing.
    """
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul

        cv2.imshow('test', data)
        cv2.waitKey(0)
    else:
        print('-NG: dcamtest_show_image(data) only support Numpy.uint16 data')


def dcam_show_single_captured_image(iDevice=0):
    """Show a single image.
    Capture and show a single image.

    Args:
        iDevice (int): Device index.

    Returns:
        Nothing.
    """
    data_list=[]
    if Dcamapi.init() is not False:
        dcam = Dcam(iDevice)
        if dcam.dev_open() is not False:
            if dcam.buf_alloc(100) is not False:
                if dcam.cap_start(bSequence=False) is not False:
                    timeout_milisec = 1000
                    start_time = time.perf_counter()
                    for i in range(0,100):
                        if dcam.wait_capevent_frameready(timeout_milisec) is not False:
                            data = dcam.buf_getlastframedata()
                            data_list.append(data)
                            time_now = time.perf_counter()
                            # if (time_now-start_time>5):
                            #     break
                            # dcamtest_show_framedata(data)
                            # break

                        # dcamerr = dcam.lasterr()
                        # if dcamerr.is_timeout():
                        #     print('===: timeout')
                        #     continue

                        # print('-NG: Dcam.wait_event() fails with error {}'.format(dcamerr))
                        # break
                else:
                    print('-NG: Dcam.cap_start() fails with error {}'.format(dcam.lasterr()))

                dcam.buf_release()
            else:
                print('-NG: Dcam.buf_alloc(1) fails with error {}'.format(dcam.lasterr()))
            dcam.dev_close()
        else:
            print('-NG: Dcam.dev_open() fails with error {}'.format(dcam.lasterr()))
    else:
        print('-NG: Dcamapi.init() fails with error {}'.format(Dcamapi.lasterr()))

    Dcamapi.uninit()
    return data_list


if __name__ == '__main__':
    data_list = dcam_show_single_captured_image()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()

In [ ]:
for data in data_list:
    if data.dtype == np.uint16:
        imax = np.amax(data)
        if imax > 0:
            imul = int(65535 / imax)
            # print('Multiple %s' % imul)
            data = data * imul
    cv2.imshow('test', data)
    cv2.waitKey(1)
cv2.destroyAllWindows()